In [1]:
#Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import re
import json
import datetime
import re
import os
from statistics import mode
import ast
import gensim
import tensorflow
from tensorflow import keras
from keras.layers import LSTM, Dense, BatchNormalization, Dropout
from keras.models import Sequential
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
#Install openpyxl for opening excel file
pip install openpyxl

In [3]:
#text pre-processing libraries

import spacy
nlp=spacy.load('en_core_web_sm')

import nltk
nltk.download("all")

In [5]:
train=pd.read_excel("../input/datadash-excel-data/Copy of DataDashTrainingData.xlsx",sheet_name="TwitterData")
test=pd.read_csv("../input/test-datadash/testing_data.csv")
senti_test=pd.read_csv("../input/senti-test/prepro_test.csv")

In [6]:
df=train.copy()

In [30]:
#Clean the data

df["chatter"].fillna("{}",inplace=True)
tweets=[];count=[]
for i in range(len(df)):
    st="";c=0
    a=eval(df["tweets"][i])
    for j in a:
        sent=j["text"]
        sent=re.sub('â€¦', '…',sent)
        sent=re.sub('â€“', '–',sent)
        sent=re.sub('â€™', '’',sent)
        sent=re.sub(r"â€[[:cntrl:]]", '”',sent)
        st+=sent
        c+=1
    tweets.append(st)
    count.append(c)
df["tweets"]=tweets
print("--done--")
chatter=[];chat_count=[]
for i in range(len(df)):
    st="";s=0
    a=eval(df["chatter"][i])
    if a=={}:
        chatter.append("")
        chat_count.append(0)
        continue
    for j in a:
        sent=j["text"]
        sent=re.sub('â€¦', '…',sent)
        sent=re.sub('â€“', '–',sent)
        sent=re.sub('â€™', '’',sent)
        sent=re.sub(r"â€[[:cntrl:]]", '”',sent)
        st+=sent
        s+=1
    chatter.append(st)
    chat_count.append(s)
df["chatter"]=chatter
print("--done--")
gc.collect()

In [ ]:
#Remove stop-words and non-ascii words

col1=[];col2=[]
for doc in nlp.pipe(df["tweets"],batch_size=500,n_process=-1):
    doc=nlp(re.sub(r"#\w+","",str(doc)))
    col1.append(" ".join([(token.text).lower() for token in doc if token.is_alpha and token.is_ascii and not token.is_stop]))
for doc in nlp.pipe(df["chatter"],batch_size=500,n_process=-1):
    doc=nlp(re.sub(r"#\w+","",str(doc)))
    col2.append(" ".join([(token.text).lower() for token in doc if token.is_alpha and token.is_ascii and not token.is_stop]))
df["tweets"]=col1
df["chatter"]=col2

In [79]:
#Training data with sentiments using the above cleaned data
senti=pd.read_csv("../input/datadash-senti/sents (1).csv")

In [80]:
df1=senti.copy()
df1.drop(["name","date"],axis=1,inplace=True)
train["chatter"].fillna("",inplace=True)

In [81]:
df1.fillna(0,inplace=True)
df1.drop(['tweet_sub','tweet_stanza_sent',"chats_sub", 'chats_stanza_sent'],axis=1,inplace=True)

In [82]:
#Clustered based on change and rank columns and added it as a feature

kmeans=KMeans(n_clusters=93,random_state=0).fit(df1.loc[:,["change","rank"]])
df1["labels"]=(kmeans.labels_)+1

In [83]:
#Check columns

df1.columns

In [84]:
X=df1.drop(["rank on the following day (dpv1)","rank 2 days later            ( dpv2)","rank 3 days later          (dpv3)"],axis=1)
Y1=df1["rank on the following day (dpv1)"]
Y2=df1["rank 2 days later            ( dpv2)"]
Y3=df1["rank 3 days later          (dpv3)"]

In [85]:
#Split into train and test

from sklearn.model_selection import train_test_split
x_train,x_test,y_train1,y_test1=train_test_split(X,Y1,test_size=0.2,random_state=0,shuffle=True)
y_train2=Y2[list(y_train1.index)]
y_train3=Y3[list(y_train1.index)]
y_test2=Y2[list(y_test1.index)]
y_test3=Y3[list(y_test1.index)]

In [86]:
#Collect garbage

gc.collect()

In [88]:
from sklearn.ensemble import GradientBoostingRegressor
gb=GradientBoostingRegressor(random_state=0)
gb.fit(x_train,y_train1)
y_pred=gb.predict(x_test)
pred=[]
for k in y_pred:
    pred.append(round(k,1))
e1=np.sqrt(mse(y_test1,pred))
print(e1)

In [89]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(random_state=0)
rf.fit(x_train,y_train1)
y_pred=rf.predict(x_test)
pred=[]
for k in y_pred:
    pred.append(round(k,1))
e1=np.sqrt(mse(y_test1,pred))
print(e1)

In [90]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
bag1=BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=9),n_estimators=2000,random_state=0)
bag1.fit(x_train,y_train1)
y_pred=bag1.predict(x_test)
pred=[]
for k in y_pred:
    pred.append(round(k,1))
e1=np.sqrt(mse(y_test1,pred))
print(e1)

bag2=BaggingRegressor(base_estimator=KNeighborsRegressor(),n_estimators=2000,random_state=0)
bag2.fit(x_train,y_train2)
y_pred=bag2.predict(x_test)
pred=[]
for k in y_pred:
    pred.append(round(k,1))
e2=np.sqrt(mse(y_test2,pred))
print(e2)

bag3=BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=3),n_estimators=2000,random_state=0)
bag3.fit(x_train,y_train3)
y_pred=bag3.predict(x_test)
pred=[]
for k in y_pred:
    pred.append(round(k,1))
e3=np.sqrt(mse(y_test3,pred))
print(e3)

In [91]:
from sklearn.ensemble import StackingRegressor
stc1=StackingRegressor([("gb",gb),("rf",rf),("bag1",bag1)])
stc1.fit(x_train,y_train1)
y_pred=stc1.predict(x_test)
pred1=[]
for i in y_pred:
    pred1.append(round(i,1))
e1=np.sqrt(mse(y_test1,pred1))
print(e1)

stc2=StackingRegressor([("gb",gb),("rf",rf),("bag2",bag2)])
stc2.fit(x_train,y_train2)
y_pred=stc2.predict(x_test)
pred2=[]
for i in y_pred:
    pred2.append(round(i,1))
e2=np.sqrt(mse(y_test2,pred2))
print(e2)

stc3=StackingRegressor([("gb",gb),("rf",rf),("bag3",bag3)])
stc3.fit(x_train,y_train3)
y_pred=stc3.predict(x_test)
pred3=[]
for i in y_pred:
    pred3.append(round(i,1))
e3=np.sqrt(mse(y_test3,pred3))
print(e3)

In [92]:
# The prediction for rank2 is taken as average of rank1 and rank3 which is better than prediction

a=(np.array(pred1)+np.array(pred3))/2
np.sqrt(mse(y_test2,a))

In [93]:
#Preprocess test data

test1=senti_test.copy()
test1.drop(['tweet_sub','tweet_stanza_sent',"chats_sub", 'chats_stanza_sent'],axis=1,inplace=True)
test1.drop(["ID"],inplace=True,axis=1)
test1.fillna(0,inplace=True)

In [94]:
#Predict the cluster labels for labels feature

test1["labels"]=(kmeans.predict(test1.loc[:,["change","rank"]]))+1
test1["labels"]=pd.Categorical(test1["labels"])

In [95]:
#Checking the columns of test and train

test1.columns==x_train.columns

In [96]:
#Predict ranks

pred1=stc1.predict(test1)
out1=[]
for i in pred1:
    out1.append(round(i,1))

pred3=stc3.predict(test1)
out3=[]
for i in pred3:
    out3.append(round(i,1))

pred2=(np.array(pred1)+np.array(pred3))/2
out2=[]
for i in pred2:
    out2.append(round(i,1))

output=pd.DataFrame()
output["ID"]=test["ID"]
output["rank1"]=out1
output["rank2"]=out2
output["rank3"]=out3
output.to_csv("sub.csv",index=False)
pd.read_csv("sub.csv")